In [3]:
# Limpando a memoria

rm(list=ls())

# Carregando Biblioteca e funcoes

library('rgdal')

source('0-functions.R')


# Definindo variaveis endereco
pathin<-'../2-Input/'
pathshp<-'/media/hellgate/16D7D0A761E2DB6A/MEGA/Shapefiles/'
pathout<-'../4-Output/'

# Lendo shapefile
map0<-readOGR(paste0(pathshp,'Br_2019/br_municipios/BR_Municipios_2019.shp'))
mapUF<-readOGR(paste0(pathshp,'Br_2019/br_unidades_da_federacao/BR_UF_2019.shp'))



Warning message in OGRSpatialRef(dsn, layer, morphFromESRI = morphFromESRI, dumpSRS = dumpSRS, :
“Discarded datum Sistema_de_Referencia_Geocentrico_para_las_AmericaS_2000 in Proj4 definition: +proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +no_defs”


OGR data source with driver: ESRI Shapefile 
Source: "/media/hellgate/16D7D0A761E2DB6A/MEGA/Shapefiles/Br_2019/br_municipios/BR_Municipios_2019.shp", layer: "BR_Municipios_2019"
with 5572 features
It has 4 fields


Warning message in OGRSpatialRef(dsn, layer, morphFromESRI = morphFromESRI, dumpSRS = dumpSRS, :
“Discarded datum Sistema_de_Referencia_Geocentrico_para_las_AmericaS_2000 in Proj4 definition: +proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +no_defs”


OGR data source with driver: ESRI Shapefile 
Source: "/media/hellgate/16D7D0A761E2DB6A/MEGA/Shapefiles/Br_2019/br_unidades_da_federacao/BR_UF_2019.shp", layer: "BR_UF_2019"
with 27 features
It has 4 fields


In [5]:

map1<-map2<-map3<-map4<-map0

map1@data[,5:16]<-map2@data[,5:16]<-map4@data[,5:16]<-NA
map3@data[,5:7]<-NA

# Listando arquivos de interesse
filesd<-list.files(pathin,pattern='cp_mes')

# Listando as Unidades Federativas
SUF<-sort(unique(map0@data[,3]))

# Criando vetor com a data
Date0<-seq(as.Date('2014-01-01'),as.Date('2022-12-31'),by='1 month')

filesd

[1] "Dengue_cp_mes_2014csv.csv" "Dengue_cp_mes_2015csv.csv"
[3] "Dengue_cp_mes_2016csv.csv" "Dengue_cp_mes_2017csv.csv"
[5] "Dengue_cp_mes_2018csv.csv" "Dengue_cp_mes_2019csv.csv"
[7] "Dengue_cp_mes_2020csv.csv" "Dengue_cp_mes_2021csv.csv"
[9] "Dengue_cp_mes_2022csv.csv"

In [6]:
for(uf in SUF){
	# Fazendo a selecao por Unidade Federativa no Shapefile
	map_rec<-subset(map0,SIGLA_UF %in% uf)
	
	# Criando objeto com o nome das colunas
	nomes<-c('Ano','Mes',map_rec@data[,1])
	
	# Criando o objeto que reunira as informacoes de saida
	output<-matrix(NA,nrow=12*length(filesd),ncol=length(nomes))
	colnames(output)<-nomes
	
	# Inserindo informacoes de ano e mes no objeto de saida
	output[,1:2]<-matrix(as.numeric(unlist(strsplit(as.character(Date0),'-'))),ncol=3,byrow=T)[,1:2]

	for(i in 1:length(filesd)){
		# Efetuando a leitura dos arquivos de interesse
		input<-read.csv(paste0(pathin,filesd[i]),header=T,sep=';',dec=',')[,c(1,3,19:30)]
		
		# Selecionando a unidade federativa no arquivo de interesse
		pos_uf<-which(input[,2]==uf)

		# Atribuindo a informacao de interesse nas devidas celulas
		input_tmp<-input[pos_uf,]

		# Identificando o ano de referencia do arquivo
		ano<-substr(filesd[i],15,18)
		
		# Atribuindo as informacoes de interesse ao seu respectivo geocodigo e ano		
		for(j in 3:length(nomes)){
			pos_mun<-which(nomes[j]==input_tmp[,1])
			pos_ano<-which(output[,1]==ano)
			output[pos_ano,j]<-as.numeric(input_tmp[pos_mun,-c(1:2)])
		}	
	}

	# Calculando o padrao medio, por mes
	output_c<-aggregate(output,by=list(output[,2]),FUN='mean',na.rm=T)[,-c(1:2)]

	# Calculando meses com maior incidencia

	output_d<-output_f<-output_c

	for(k in 2:NCOL(output_c)){
		output_d[,k]<-ifelse(output_c[,k]>=mean(output_c[,k]) & any(output_c[,k]!=0),output_c[,1],0)
		output_f[,k]<-Id_cycle(output_d[,k],12,2,k)}

	output2<-output
	# Frequencia
	for(m in 3:NCOL(output)){
		output2[,m]<-as.numeric(output[,m]>=mean(output[,m],na.rm=T) & mean(output[,m],na.rm=T)!=0)
	}

	output_cf<-aggregate(output2,by=list(output2[,2]),FUN='sum',na.rm=T)[,-c(2:3)]

	pos0<-which(map1@data[,3]==uf)

	# Gerando espacializacao
	for(l in 1:length(pos0)){
		map1@data[pos0[l],-c(1:4)]<-output_f[,(l+1)]
		map4@data[pos0[l],-c(1:4)]<-output_cf[,(l+1)]
		pos1<-which(output_f[,(l+1)]!=0)
		{if(length(pos1)!=0)
			{map2@data[pos0[l],c(pos1+4)]<-output_c[pos1,(l+1)]
			NUM<-sum(map2@data[pos0[l],c(pos1+4)],na.rm=T)
			map3@data[pos0[l],5]<-NUM}
		}
	}

	
	# Escrevendo as informacoes concatenadas
	write.table(output,file=paste0(pathout,'1-Mensal/Taxa_Inc_Dengue_2014_2022_UF_',uf,'.csv'),quote=F,sep=';',dec='.',col.names=T,row.names=F)
	
	# Escrevendo as informacoes medias
	write.table(output_c,file=paste0(pathout,'2-Clima/Media_Mensal_Taxa_Inc_Dengue_2014_2022_UF_',uf,'.csv'),quote=F,sep=';',dec='.',col.names=T,row.names=F)

	# Escrevendo os meses com ocorrencia acima da media
	write.table(output_f,file=paste0(pathout,'2-Clima/Meses_Maximos_UF_',uf,'.csv'),quote=F,sep=';',dec='.',col.names=T,row.names=F)

	print(uf)
}


[1] "AC"
[1] "AL"
[1] "AM"
[1] "AP"
[1] "BA"
[1] "CE"
[1] "DF"
[1] "ES"
[1] "GO"
[1] "MA"
[1] "MG"
[1] "MS"
[1] "MT"
[1] "PA"
[1] "PB"
[1] "PE"
[1] "PI"
[1] "PR"
[1] "RJ"
[1] "RN"
[1] "RO"
[1] "RR"
[1] "RS"
[1] "SC"
[1] "SE"
[1] "SP"
[1] "TO"
